# ParaToric: Create SLURM job scripts for extended toric code QMC 

This notebook is used to create SLURM job scripts to run continuous-time QMC simulations of the extended toric code
$$
\hat{\mathcal{H}}_{\mathrm{eTC}} = - \mu \sum_+ \prod_{l \in +} \hat{\tau}_l^x - J \sum_{\square} \prod_{l \in \square} \hat{\tau}_l^z - h \sum_{l} \hat{\tau}_l^x - \lambda \sum_{l} \hat{\tau}_l^z,
$$
where $J, \lambda \geq 0$ in the $\hat{\tau}^x$-basis and $\mu, h \geq 0$ in the $\hat{\tau}^z$-basis. The temperature $T=1 / \beta$ is finite. The algorithm is based on [Wu <em>et al.</em>, Phys. Rev. B <strong>85</strong>, 195104 (2012)](https://doi.org/10.1103/PhysRevB.85.195104).

## User-specific data

In [ ]:
EMAIL = "your.email@uni.com"
PARATORICDIR = "/path/to/ParaToric"
LOGDIR = "where/log/files/are/stored"
OUTDIR = "/where/output/files/are/stored"

MODULES = '''\
module load spack/2024.07
module load hdf5/1.14.3-gcc-13.2.0-openmpi
module load python/3.12-2024.10 
module load boost/1.85.0-gcc-13.2.0
module load gcc/13.2.0 
export MKL_NUM_THREADS=1
'''

## $T$-sweep

In [ ]:
import os

TIME = '3-00:00:00'  # Format: DAYS-HOURS:MINUTES:SECONDS
NTASKS = '30'
PROCESSES = '30'
MEMORY = '16GB'
SIMULATION_TYPE = 'etc_T_sweep'
T_LOWER = '0.05'
T_UPPER = '5.0'
T_STEPS = '30'
MU_CONST = '1'
J_CONST = '1' 
H_CONST = '0.2' 
H_CONST_THERM = H_CONST
LMBDA_CONST = '0.8'
LMBDA_CONST_THERM = LMBDA_CONST
N_RESAMPLES = '1000'
CUSTOM_THERM = '0'
OBSERVABLES = 'percolation_strength percolation_probability plaquette_percolation_strength plaquette_percolation_probability largest_cluster largest_plaquette_cluster string_number energy energy_h energy_mu energy_J energy_lmbda sigma_x sigma_z star_x plaquette_z staggered_imaginary_times delta anyon_count anyon_density fredenhagen_marcu sigma_x_susceptibility sigma_x_dynamical_susceptibility sigma_z_susceptibility sigma_z_dynamical_susceptibility'
SEED = '0' # 0 means random seed
BASIS = 'z'
DIMENSIONALITY = '2'
LATTICE = 'square'
SYSTEM_SIZE = '10'
BOUNDARIES = 'periodic'
DEFAULT_SPIN = '1'
if BASIS == 'x':
    N_THERMALIZATION = f'{500 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(T_LOWER)), 20) * max(int(float(LMBDA_CONST)), int(float(J_CONST)))}'
else:
    N_THERMALIZATION = f'{500 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(T_LOWER)), 20) * max(int(float(H_CONST)), int(float(MU_CONST)))}'
N_BETWEEN_SAMPLES = f'{8 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(T_LOWER)), 20)}'
N_SAMPLES = '10000'
SNAPSHOTS = '1'
FULL_TIME_SERIES = '0'

JOB_NAME = f'Q_T_{SYSTEM_SIZE}_{LMBDA_CONST}_{H_CONST}_{BASIS}'
COMMENT = 'QMC temperature-sweep for extended toric code'

JOB_SCRIPT_NAME = f'eTC_T_sweep_{LATTICE}_{BASIS}_{SYSTEM_SIZE}_{BOUNDARIES}_{LMBDA_CONST}_{H_CONST}.sh'

PYTHON_COMMAND = f'python3 -u ./python/cli/paratoric.py -sim {SIMULATION_TYPE} -Ns {N_SAMPLES} -Nbs {N_BETWEEN_SAMPLES} ' \
                 f'-Nth {N_THERMALIZATION} -Tl {T_LOWER} -Tu {T_UPPER} -Ts {T_STEPS} -muc {MU_CONST} ' \
                 f'-hc {H_CONST} -hct {H_CONST_THERM} -Jc {J_CONST} -lmbdac {LMBDA_CONST} -lmbdact {LMBDA_CONST_THERM} -Nr {N_RESAMPLES} -cth {CUSTOM_THERM} -obs {OBSERVABLES} -s {SEED} ' \
                 f'-bas {BASIS} -lat {LATTICE} -L {SYSTEM_SIZE} -bound {BOUNDARIES} ' \
                 f'-dsp {DEFAULT_SPIN} -proc {PROCESSES} -snap {SNAPSHOTS} -fts {FULL_TIME_SERIES} ' \
                 f'-outdir {OUTDIR}'

with open (JOB_SCRIPT_NAME, 'w') as rsh:
    rsh.write(f'''\
#!/bin/bash
#SBATCH --job-name={JOB_NAME}
#SBATCH --comment="{COMMENT}"
#SBATCH --time={TIME}
#SBATCH --mail-type=ALL
#SBATCH --mail-user={EMAIL}
#SBATCH --chdir={PARATORICDIR}
#SBATCH --output={os.path.join(LOGDIR, "oslurm.%j.%N.out")}
#SBATCH --ntasks={NTASKS}
#SBATCH --nodes=1
#SBATCH --constraint=x86-64-v3
#SBATCH --mem={MEMORY}
#SBATCH --partition=cluster

{MODULES}
{PYTHON_COMMAND}
''')

## $h$-sweep ($\hat{\tau}^x$-field)

In [ ]:
import os

TIME = '3-00:00:00'  # Format: DAYS-HOURS:MINUTES:SECONDS
NTASKS = '64'
PROCESSES = '64'
MEMORY = '16GB'
SIMULATION_TYPE = 'etc_h_sweep'
H_LOWER = '0.2'
H_UPPER = '0.4'
H_STEPS = '64'
H_CONST_THERM = '0.0'
MU_CONST = '1'
J_CONST = '1' 
LMBDA_CONST = '0.2'
LMBDA_CONST_THERM = LMBDA_CONST
N_RESAMPLES = '1000'
CUSTOM_THERM = '0'
OBSERVABLES = 'percolation_strength percolation_probability plaquette_percolation_strength plaquette_percolation_probability largest_cluster largest_plaquette_cluster string_number energy energy_h energy_mu energy_J energy_lmbda sigma_x sigma_z star_x plaquette_z staggered_imaginary_times delta anyon_count anyon_density fredenhagen_marcu sigma_x_susceptibility sigma_x_dynamical_susceptibility sigma_z_susceptibility sigma_z_dynamical_susceptibility'
SEED = '0' # 0 means random seed
BASIS = 'z'
DIMENSIONALITY = '2'
LATTICE = 'square'
SYSTEM_SIZE = 40
TEMPERATURE = f'{1/SYSTEM_SIZE}'
SYSTEM_SIZE = str(SYSTEM_SIZE)
BOUNDARIES = 'periodic'
DEFAULT_SPIN = '1'
if BASIS == 'x':
    N_THERMALIZATION = f'{500 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20) * max(int(float(LMBDA_CONST)), int(float(J_CONST)))}'
else:
    N_THERMALIZATION = f'{500 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20) * max(int(float(H_UPPER)), int(float(MU_CONST)))}'
N_BETWEEN_SAMPLES = f'{8 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20)}'
N_SAMPLES = '30000'
SNAPSHOTS = '0'
FULL_TIME_SERIES = '0'

JOB_NAME = f'Q_h_{SYSTEM_SIZE}_{LMBDA_CONST}_{TEMPERATURE}_{BASIS}'
COMMENT = 'QMC h-sweep for extended toric code'

JOB_SCRIPT_NAME = f'eTC_h_sweep_{LATTICE}_{BASIS}_{SYSTEM_SIZE}_{BOUNDARIES}_{TEMPERATURE}_{LMBDA_CONST}.sh'

PYTHON_COMMAND = f'python3 -u ./python/cli/paratoric.py -sim {SIMULATION_TYPE} -Ns {N_SAMPLES} -Nbs {N_BETWEEN_SAMPLES} ' \
                 f'-Nth {N_THERMALIZATION} -T {TEMPERATURE} -muc {MU_CONST} ' \
                 f'-hl {H_LOWER} -hu {H_UPPER} -hs {H_STEPS} -hct {H_CONST_THERM} -Jc {J_CONST} -lmbdac {LMBDA_CONST} -lmbdact {LMBDA_CONST_THERM} -Nr {N_RESAMPLES} -cth {CUSTOM_THERM} -obs {OBSERVABLES} -s {SEED} ' \
                 f'-bas {BASIS} -lat {LATTICE} -L {SYSTEM_SIZE} -bound {BOUNDARIES} ' \
                 f'-dsp {DEFAULT_SPIN} -proc {PROCESSES} -snap {SNAPSHOTS} -fts {FULL_TIME_SERIES} ' \
                 f'-outdir {OUTDIR}'

with open (JOB_SCRIPT_NAME, 'w') as rsh:
    rsh.write(f'''\
#!/bin/bash
#SBATCH --job-name={JOB_NAME}
#SBATCH --comment="{COMMENT}"
#SBATCH --time={TIME}
#SBATCH --mail-type=ALL
#SBATCH --mail-user={EMAIL}
#SBATCH --chdir={PARATORICDIR}
#SBATCH --output={os.path.join(LOGDIR, "oslurm.%j.%N.out")}
#SBATCH --ntasks={NTASKS}
#SBATCH --nodes=1
#SBATCH --constraint=x86-64-v3
#SBATCH --mem={MEMORY}
#SBATCH --partition=cluster

{MODULES}
{PYTHON_COMMAND}
''')

## $\lambda$-sweep ($\hat{\tau}^z$-field)

In [ ]:
import os

TIME = '2-00:00:00'  # Format: DAYS-HOURS:MINUTES:SECONDS
NTASKS = '21'
PROCESSES = '21'
MEMORY = '16GB'
SIMULATION_TYPE = 'etc_lmbda_sweep'
LMBDA_LOWER = '0.1'
LMBDA_UPPER = '0.7'
LMBDA_STEPS = '21'
LMBDA_CONST_THERM = '0.0'
MU_CONST = '1'
J_CONST = '1' 
H_CONST = '0.1'
H_CONST_THERM = H_CONST
N_RESAMPLES = '1000'
CUSTOM_THERM = '0'
OBSERVABLES = 'percolation_strength percolation_probability plaquette_percolation_strength plaquette_percolation_probability largest_cluster largest_plaquette_cluster string_number energy energy_h energy_mu energy_J energy_lmbda sigma_x sigma_z star_x plaquette_z staggered_imaginary_times delta anyon_count anyon_density fredenhagen_marcu sigma_x_susceptibility sigma_x_dynamical_susceptibility sigma_z_susceptibility sigma_z_dynamical_susceptibility'
SEED = '0' # 0 means random seed
BASIS = 'x'
DIMENSIONALITY = '2'
LATTICE = 'triangular'
SYSTEM_SIZE = 20
TEMPERATURE = f'{1/SYSTEM_SIZE}'
SYSTEM_SIZE = str(SYSTEM_SIZE)
BOUNDARIES = 'periodic'
DEFAULT_SPIN = '1'
if BASIS == 'x':
    N_THERMALIZATION = f'{500 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20) * max(int(float(LMBDA_UPPER)), int(float(J_CONST)))}'
else:
    N_THERMALIZATION = f'{500 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20) * max(int(float(H_CONST)), int(float(MU_CONST)))}'
N_BETWEEN_SAMPLES = f'{8 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20)}'
N_SAMPLES = '10000'
SNAPSHOTS = '0'
FULL_TIME_SERIES = '0'

JOB_NAME = f'Q_l_{SYSTEM_SIZE}_{H_CONST}_{TEMPERATURE}_{BASIS}'
COMMENT = 'QMC lmbda-sweep for extended toric code'

JOB_SCRIPT_NAME = f'eTC_lmbda_sweep_{LATTICE}_{BASIS}_{SYSTEM_SIZE}_{BOUNDARIES}_{TEMPERATURE}_{H_CONST}.sh'

PYTHON_COMMAND = f'python3 -u ./python/cli/paratoric.py -sim {SIMULATION_TYPE} -Ns {N_SAMPLES} -Nbs {N_BETWEEN_SAMPLES} ' \
                 f'-Nth {N_THERMALIZATION} -T {TEMPERATURE} -muc {MU_CONST} ' \
                 f'-lmbdal {LMBDA_LOWER} -lmbdau {LMBDA_UPPER} -lmbdas {LMBDA_STEPS} -lmbdact {LMBDA_CONST_THERM} -Jc {J_CONST} -hc {H_CONST} -hct {H_CONST_THERM} -Nr {N_RESAMPLES} -cth {CUSTOM_THERM} -obs {OBSERVABLES} -s {SEED} ' \
                 f'-bas {BASIS} -lat {LATTICE} -L {SYSTEM_SIZE} -bound {BOUNDARIES} ' \
                 f'-dsp {DEFAULT_SPIN} -proc {PROCESSES} -snap {SNAPSHOTS} -fts {FULL_TIME_SERIES} ' \
                 f'-outdir {OUTDIR}'

with open (JOB_SCRIPT_NAME, 'w') as rsh:
    rsh.write(f'''\
#!/bin/bash
#SBATCH --job-name={JOB_NAME}
#SBATCH --comment="{COMMENT}"
#SBATCH --time={TIME}
#SBATCH --mail-type=ALL
#SBATCH --mail-user={EMAIL}
#SBATCH --chdir={PARATORICDIR}
#SBATCH --output={os.path.join(LOGDIR, "oslurm.%j.%N.out")}
#SBATCH --ntasks={NTASKS}
#SBATCH --nodes=1
#SBATCH --constraint=x86-64-v3
#SBATCH --mem={MEMORY}
#SBATCH --partition=cluster

{MODULES}
{PYTHON_COMMAND}
''')

## $\circ$-sweep 

In [ ]:
import numpy as np
import os

TIME = '1-00:00:00' # Format: DAYS-HOURS:MINUTES:SECONDS
NTASKS = '32'
PROCESSES = '30'
MEMORY = '16GB'
SIMULATION_TYPE = 'etc_circle_sweep'
RADIUS = '2.0'
THETA_LOWER = f'0'
THETA_UPPER = f'{np.pi/2}'
THETA_STEPS = '30'
MU_CONST = '1'
J_CONST = '1' 
H_CONST = '0.0'
LMBDA_CONST = '0.0'
N_RESAMPLES = '1000'
OBSERVABLES = 'percolation_strength percolation_probability plaquette_percolation_strength plaquette_percolation_probability largest_cluster largest_plaquette_cluster string_number energy energy_h energy_mu energy_J energy_lmbda sigma_x sigma_z star_x plaquette_z staggered_imaginary_times delta anyon_count anyon_density fredenhagen_marcu sigma_x_susceptibility sigma_x_dynamical_susceptibility sigma_z_susceptibility sigma_z_dynamical_susceptibility'
SEED = '0' # 0 means random seed
BASIS = 'z'
DIMENSIONALITY = '2'
LATTICE = 'square'
SYSTEM_SIZE = 10
TEMPERATURE = f'{1/SYSTEM_SIZE}'
SYSTEM_SIZE = str(SYSTEM_SIZE)
BOUNDARIES = 'periodic'
DEFAULT_SPIN = '1'
if BASIS == 'x':
    N_THERMALIZATION = f'{500 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20) * max(int(float(LMBDA_CONST)+float(RADIUS)), int(float(J_CONST)))}'
else:
    N_THERMALIZATION = f'{500 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20) * max(int(float(H_CONST)+float(RADIUS)), int(float(MU_CONST)))}'
N_BETWEEN_SAMPLES = f'{8 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20)}'
N_SAMPLES = '10000'
FULL_TIME_SERIES = '0'

JOB_NAME = f'Q_c_{SYSTEM_SIZE}_{LMBDA_CONST}_{H_CONST}_{RADIUS}_{TEMPERATURE}_{BASIS}'
COMMENT = 'QMC circle-sweep for extended toric code'

JOB_SCRIPT_NAME = f'eTC_circle_sweep_{LATTICE}_{BASIS}_{SYSTEM_SIZE}_{BOUNDARIES}_{TEMPERATURE}_{H_CONST}_{LMBDA_CONST}_{RADIUS}_{THETA_STEPS}.sh'

PYTHON_COMMAND = f'python3 -u ./python/cli/paratoric.py -sim {SIMULATION_TYPE} -Ns {N_SAMPLES} -Nbs {N_BETWEEN_SAMPLES} ' \
                 f'-Nth {N_THERMALIZATION} -T {TEMPERATURE} -muc {MU_CONST} ' \
                 f'-lmbdac {LMBDA_CONST} -rad {RADIUS} -Thl {THETA_LOWER} -Thu {THETA_UPPER} -Ths {THETA_STEPS} -Jc {J_CONST} -hc {H_CONST} -Nr {N_RESAMPLES} -obs {OBSERVABLES} -s {SEED} ' \
                 f'-bas {BASIS} -lat {LATTICE} -L {SYSTEM_SIZE} -bound {BOUNDARIES} ' \
                 f'-dsp {DEFAULT_SPIN} -proc {PROCESSES} -fts {FULL_TIME_SERIES} ' \
                 f'-outdir {OUTDIR}'

with open (JOB_SCRIPT_NAME, 'w') as rsh:
    rsh.write(f'''\
#!/bin/bash
#SBATCH --job-name={JOB_NAME}
#SBATCH --comment="{COMMENT}"
#SBATCH --time={TIME}
#SBATCH --mail-type=ALL
#SBATCH --mail-user={EMAIL}
#SBATCH --chdir={PARATORICDIR}
#SBATCH --output={os.path.join(LOGDIR, "oslurm.%j.%N.out")}
#SBATCH --ntasks={NTASKS}
#SBATCH --nodes=1
#SBATCH --constraint=x86-64-v3
#SBATCH --mem={MEMORY}
#SBATCH --partition=cluster

{MODULES}
{PYTHON_COMMAND}
''')

## Thermalization

In [ ]:
import os

TIME = '1-00:00:00' # Format: DAYS-HOURS:MINUTES:SECONDS
NTASKS = '16'
PROCESSES = '15'
MEMORY = '16GB'
SIMULATION_TYPE = 'etc_thermalization'
TEMPERATURE = '0.1'
H_CONST = '0.3' 
MU_CONST = '1'
J_CONST = '1' 
LMBDA_CONST = '0.4' 
N_RESAMPLES = '1000'
OBSERVABLES = 'percolation_strength percolation_probability plaquette_percolation_strength plaquette_percolation_probability largest_cluster largest_plaquette_cluster string_number energy energy_h energy_mu energy_J energy_lmbda sigma_x sigma_z star_x plaquette_z staggered_imaginary_times delta anyon_count anyon_density fredenhagen_marcu sigma_x_susceptibility sigma_x_dynamical_susceptibility sigma_z_susceptibility sigma_z_dynamical_susceptibility'
SEED = '0' # 0 means random seed
BASIS = 'x'
DIMENSIONALITY = '2'
LATTICE = 'square'
SYSTEM_SIZE = '10'
BOUNDARIES = 'periodic'
DEFAULT_MATTER_OCCUPATION = '0'
DEFAULT_SPIN = '1'
if BASIS == 'x':
    N_THERMALIZATION = f'{500 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20) * max(int(float(LMBDA_CONST)), int(float(J_CONST)))}'
else:
    N_THERMALIZATION = f'{500 * int(SYSTEM_SIZE)**int(DIMENSIONALITY) * max(int(1/float(TEMPERATURE)), 20) * max(int(float(H_CONST)), int(float(MU_CONST)))}'
REPETITIONS = '10'

JOB_NAME = f'Q_th_{SYSTEM_SIZE}_{LMBDA_CONST}_{H_CONST}_{TEMPERATURE}_{BASIS}'
COMMENT = 'QMC thermalization for extended toric code'

JOB_SCRIPT_NAME = f'eTC_therm_{LATTICE}_{BASIS}_{SYSTEM_SIZE}_{BOUNDARIES}_{TEMPERATURE}_{LMBDA_CONST}_{H_CONST}.sh'

PYTHON_COMMAND = f'python3 -u ./python/cli/paratoric.py -sim {SIMULATION_TYPE} ' \
                 f'-Nth {N_THERMALIZATION} -reps {REPETITIONS} -T {TEMPERATURE} -muc {MU_CONST} ' \
                 f'-hc {H_CONST} -Jc {J_CONST} -lmbdac {LMBDA_CONST} -Nr {N_RESAMPLES} -obs {OBSERVABLES} -s {SEED} ' \
                 f'-bas {BASIS} -lat {LATTICE} -L {SYSTEM_SIZE} -bound {BOUNDARIES} ' \
                 f'-dsp {DEFAULT_SPIN} -proc {PROCESSES} ' \
                 f'-outdir {OUTDIR}'

with open (JOB_SCRIPT_NAME, 'w') as rsh:
    rsh.write(f'''\
#!/bin/bash
#SBATCH --job-name={JOB_NAME}
#SBATCH --comment="{COMMENT}"
#SBATCH --time={TIME}
#SBATCH --mail-type=ALL
#SBATCH --mail-user={EMAIL}
#SBATCH --chdir={PARATORICDIR}
#SBATCH --output={os.path.join(LOGDIR, "oslurm.%j.%N.out")}
#SBATCH --ntasks={NTASKS}
#SBATCH --nodes=1
#SBATCH --constraint=x86-64-v3
#SBATCH --mem={MEMORY}
#SBATCH --partition=cluster

{MODULES}
{PYTHON_COMMAND}
''')